# 📊 Xem Dữ Liệu Silver Layer - OER Lakehouse

Notebook này giúp bạn xem và phân tích dữ liệu trong Silver layer một cách nhanh chóng.

**Ngày tạo:** 21/10/2025  
**Mục đích:** Query và explore dữ liệu OER từ Iceberg tables

## 🔧 Bước 1: Khởi tạo Spark + Iceberg

In [1]:


from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Stop existing session nếu có
try:
    spark.stop()
    print("🔄 Đã dừng Spark session cũ")
except:
    pass

print("📦 Đang khởi tạo Spark với Iceberg...")

spark = SparkSession.builder \
    .appName("View-Silver-Data") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.silver", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.silver.type", "hadoop") \
    .config("spark.sql.catalog.silver.warehouse", "s3a://oer-lakehouse/silver/") \
    .config("spark.sql.catalog.lakehouse", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.lakehouse.type", "hadoop") \
    .config("spark.sql.catalog.lakehouse.warehouse", "s3a://oer-lakehouse/warehouse/") \
    .config("spark.sql.defaultCatalog", "silver") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✅ Spark + Iceberg đã sẵn sàng!")
print(f"📦 Spark version: {spark.version}")

📦 Đang khởi tạo Spark với Iceberg...
✅ Spark + Iceberg đã sẵn sàng!
📦 Spark version: 3.5.0


## 📋 Bước 2: Liệt kê tất cả các bảng

In [11]:
# Xem tất cả các bảng trong Silver layer
print("📊 Danh sách các bảng trong Silver layer:")
tables_df = spark.sql("SHOW TABLES FROM silver.default")
tables_df.show(truncate=False)

# Đếm số lượng bảng
table_count = tables_df.count()
print(f"\n✅ Tổng số bảng: {table_count}")

📊 Danh sách các bảng trong Silver layer:
+---------+-------------------------------+-----------+
|namespace|tableName                      |isTemporary|
+---------+-------------------------------+-----------+
|default  |reference_faculties            |false      |
|default  |reference_subjects             |false      |
|default  |oer_resources                  |false      |
|default  |reference_program_subject_links|false      |
|default  |reference_programs             |false      |
+---------+-------------------------------+-----------+


✅ Tổng số bảng: 5


## 📚 Bước 3: Xem bảng OER Documents (DC Metadata)

In [34]:
# Cell: Query MIT OCW và OpenStax data
from pyspark.sql import functions as F

# 2. Sample MIT OCW
print("\n" + "=" * 80)
print("2. SAMPLE DỮ LIỆU MIT OCW")
print("=" * 80)
spark.sql("""
  SELECT 
    r.resource_id,
    r.title,
    r.source_system,
    r.source_url,
    r.matched_subjects,
    r.program_ids,
    r.data_quality_score
FROM silver.default.oer_resources r
WHERE ARRAY_CONTAINS(r.program_ids, 11)
  AND ARRAY_SIZE(r.matched_subjects) > 0
ORDER BY r.data_quality_score DESC
LIMIT 10;
""").show(truncate=False)



2. SAMPLE DỮ LIỆU MIT OCW
+--------------------------------+---------------------------------------------+-------------+---------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+------------

In [12]:
# Xem 10 bản ghi mẫu
print("📄 10 BẢN GHI MẪU:")
print("=" * 60)
df_documents.select(
    "*"
).show(30, truncate=80) 

📄 10 BẢN GHI MẪU:


AttributeError: 'DataFrame' object has no attribute 'query'

## 📊 Bước 4: Thống kê dữ liệu

In [ ]:
# Thống kê theo source_system
print("📊 PHÂN BỐ THEO NGUỒN DỮ LIỆU:")
print("=" * 60)
df_documents.groupBy("source_system") \
    .agg(
        F.count("*").alias("số_lượng"),
        F.countDistinct("dc_publisher").alias("số_publisher")
    ) \
    .orderBy("số_lượng", ascending=False) \
    .show(truncate=False)

In [ ]:
# Thống kê theo subject_category_code
print("📚 PHÂN BỐ THEO DANH MỤC:")
print("=" * 60)
df_documents.groupBy("subject_category_code") \
    .agg(F.count("*").alias("số_lượng")) \
    .orderBy("số_lượng", ascending=False) \
    .show(20, truncate=False)

In [ ]:
# Thống kê theo ngôn ngữ
print("🌍 PHÂN BỐ THEO NGÔN NGỮ:")
print("=" * 60)
df_documents.groupBy("dc_language") \
    .agg(F.count("*").alias("số_lượng")) \
    .orderBy("số_lượng", ascending=False) \
    .show(10, truncate=False)

## 🔍 Bước 5: Tìm kiếm và lọc dữ liệu

In [ ]:
# Tìm kiếm theo từ khóa trong title
keyword = "machine learning"  # Thay đổi từ khóa tại đây

print(f"🔍 TÌM KIẾM: '{keyword}'")
print("=" * 60)

search_results = df_documents.filter(
    F.lower(F.col("dc_title")).contains(keyword.lower())
)

print(f"📊 Tìm thấy: {search_results.count()} kết quả")
print("\n📄 Kết quả tìm kiếm:")

search_results.select(
    "dc_identifier",
    "dc_title",
    "dc_publisher",
    "source_system"
).show(20, truncate=100)

In [ ]:
# Lọc theo source_system cụ thể
source = "MIT_OCW"  # Có thể thay bằng OPENSTAX hoặc OTL

print(f"🎓 DỮ LIỆU TỪ NGUỒN: {source}")
print("=" * 60)

filtered_df = df_documents.filter(F.col("source_system") == source)

print(f"📊 Số lượng: {filtered_df.count():,} bản ghi")

filtered_df.select(
    "dc_identifier",
    "dc_title",
    "dc_publisher",
    "dc_language"
).show(15, truncate=80)

## 🏷️ Bước 6: Xem bảng Subjects

In [ ]:
# Load bảng oer_subjects
df_subjects = spark.table("lakehouse.silver.oer_subjects")

print("📊 THÔNG TIN BẢNG OER_SUBJECTS")
print("=" * 60)
print(f"📈 Tổng số bản ghi: {df_subjects.count():,}")
print("\n🔍 Schema:")
df_subjects.printSchema()

print("\n📄 Dữ liệu mẫu:")
df_subjects.show(15, truncate=False)

## 📝 Bước 7: Xem bảng Creators (Tác giả)

In [ ]:
# Load bảng oer_creators
df_creators = spark.table("lakehouse.silver.oer_creators")

print("📊 THÔNG TIN BẢNG OER_CREATORS")
print("=" * 60)
print(f"📈 Tổng số bản ghi: {df_creators.count():,}")
print("\n🔍 Schema:")
df_creators.printSchema()

print("\n📄 Top 20 tác giả:")
df_creators.groupBy("creator_name") \
    .agg(F.count("*").alias("số_tài_liệu")) \
    .orderBy("số_tài_liệu", ascending=False) \
    .show(20, truncate=False)

## 🔗 Bước 8: JOIN nhiều bảng để xem thông tin đầy đủ

In [ ]:
# JOIN documents với subjects
print("🔗 KẾT HỢP DỮ LIỆU TÀI LIỆU VÀ CHỦ ĐỀ")
print("=" * 60)

joined_df = df_documents.alias("doc") \
    .join(
        df_subjects.alias("sub"),
        F.col("doc.dc_identifier") == F.col("sub.dc_identifier"),
        "left"
    ) \
    .select(
        "doc.dc_identifier",
        "doc.dc_title",
        "doc.dc_publisher",
        "sub.subject_name",
        "sub.subject_code",
        "doc.source_system"
    )

print(f"📊 Số bản ghi sau khi join: {joined_df.count():,}")
print("\n📄 Dữ liệu mẫu:")
joined_df.show(15, truncate=80)

In [ ]:
# Thống kê các subject phổ biến nhất
print("📚 TOP 20 CHỦ ĐỀ PHỔ BIẾN NHẤT")
print("=" * 60)

df_subjects.groupBy("subject_name") \
    .agg(F.count("*").alias("số_tài_liệu")) \
    .orderBy("số_tài_liệu", ascending=False) \
    .show(20, truncate=False)

## 💾 Bước 9: Export dữ liệu (Optional)

In [ ]:
# Export một sample ra CSV để xem dễ hơn
output_path = "/opt/airflow/data/silver_sample.csv"

print(f"💾 Đang export dữ liệu mẫu ra: {output_path}")

df_documents.limit(100) \
    .select(
        "dc_identifier",
        "dc_title",
        "dc_publisher",
        "dc_language",
        "source_system",
        "subject_category_code"
    ) \
    .coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(output_path)

print("✅ Export hoàn tất!")

## 🎉 Hoàn thành!

Bạn đã xem xong dữ liệu Silver layer. Một số điều có thể làm tiếp:

1. ✏️ **Sửa các cell tìm kiếm** để tìm dữ liệu theo ý muốn
2. 📊 **Thêm visualizations** với matplotlib hoặc plotly
3. 🔍 **Phân tích sâu hơn** về quality, completeness của dữ liệu
4. 🚀 **Chạy Gold layer** để tạo các bảng phân tích

**Happy querying! 🎯**